In [ ]:
if "running_all" not in globals():
    from ipywidgets import widgets
    matrices = ["ex10", "msc04515", "s1rmq4m1", "Na5", "bcsstk18",
                "vibrobox", "cbuckle", "Pres_Poisson", "raefsky4", "vanbody",
                "ct20stif", "cant", "bcircuit", "apache1", "consph"]
    b = widgets.Button(description="Run over all matrices", button_style="success")
    output = widgets.Output()

    display(b, output)

    def run_over_all_matrices(button):
        global running_all
        global matrix
        running_all = True
        with output:
            for matrix in matrices:
                print(f"Running {matrix}...")
                %run ./overhead_analysis.ipynb # will output at this cell rather than later
            print("Finished!")
    b.on_click(run_over_all_matrices)

In [ ]:
if "running_all" not in globals():
    matrix = "bcsstk18"  # manually set to run over one matrix
matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import os
from json import load
sys.path.append(os.path.join(os.getcwd(), os.pardir))
from io_utils import load_models, load_matrices_from_dir

In [ ]:
ylims = {
    "apache1": (0, 0.3),
    "bcircuit": (0, 0.6),
    "bcsstk18": (0.2, 1),
    "cant": (0.8, 1),
    "cbuckle": (0, 0.6),
    "consph": (0, 0.02),
    "ct20stif": (0, 1),
    "ex10": (0.4, 0.9),
    "msc04515": (0, 0.5),
    "Na5": (0, 1),
    "Pres_Poisson": (0, 0.6),
    "raefsky4": (0, 0.2),
    "s1rmq4m1": (0, 0.6),
    "vanbody": (0.925, 1),
    "vibrobox": (0.4, 0.65),
}

TESTING_DATA_SIZE = 1000

df = pd.read_csv(f"data/{matrix}_{TESTING_DATA_SIZE}.csv")
errorfree_iterations = df["errorfree_iterations"][0]  # all the same
n_rows = df["n_rows"][0]  # all the same
df.head()

In [ ]:
chosen_models = {
    "apache1": "XGBRegressor",
    "bcircuit": "XGBRegressor",
    "bcsstk18": "XGBRegressor",
    "cant": "KNeighborsRegressor",
    "cbuckle": "KNeighborsRegressor",
    "consph": "Ridge",
    "ct20stif": "XGBRegressor",
    "ex10": "RandomForestRegressor",
    "msc04515": "RandomForestRegressor",
    "Na5": "XGBRegressor",
    "Pres_Poisson": "XGBRegressor",
    "raefsky4": "RandomForestRegressor",
    "s1rmq4m1": "RandomForestRegressor",
    "vanbody": "RandomForestRegressor",
    "vibrobox": "KNeighborsRegressor",
}

model_path = f"./models/{matrix}/best_{chosen_models[matrix]}.pkl"
model = load_models([model_path])[0]
model

In [ ]:
X = df[["error_iter", "pos_2norm"]].to_numpy()
df["prot_score"] = model.predict(X)
df.head()

In [ ]:
mats = load_matrices_from_dir("../matrices/raw", subset=[matrix])
mat = list(mats.values())[0]
mat

In [ ]:
with open(f"../matrices/2norms/{matrix}_pos_2norms.json") as f:
    pos_2norms = load(f)

In [ ]:
df_preds = pd.DataFrame([[i, pos_2norms[str(pos)], pos] for pos in range(mat.shape[0])
                         for i in range(errorfree_iterations)], columns=["i", "2norm", "rowid"])

In [ ]:
df_preds["output"] = model.predict(df_preds[["i", "2norm"]])
df_preds.head()

In [ ]:
ps = [1/98] + list(np.arange(0.02, 1.01, 0.01))
percentages = np.arange(0.01, 1.0, 0.01)
nonerror_runs_by_p = {p: int((len(df) / p) - len(df)) for p in ps}
max_nonerror_runs = int((len(df) / min(ps)) - len(df))
solve_iterations = np.append(df["solve_iterations"], [errorfree_iterations] * max_nonerror_runs)
slowdowns = np.append(df["slowdown"], [1] * max_nonerror_runs)
errorfree_op_count = errorfree_iterations * n_rows

In [ ]:
def compute_overheads(error_iterations, n_protections):
    return ((error_iterations * n_rows + n_protections) - errorfree_op_count) / errorfree_op_count

def protect(error_iterations, protections):
    return np.vectorize(lambda i: error_iterations[i] if not protections[i]
                        else errorfree_iterations)(range(len(error_iterations)))

def make_p_overhead_dataframe(ohs_by_p):
    return pd.concat([pd.DataFrame({"p": [ps[i]] * len(os), "overhead": os})
                         for i, os in enumerate(ohs_by_p)], ignore_index=True)

In [ ]:
nonprot_overheads_by_p = []

for p in ps:
    # need to pad dataset to add non-error runs
    n_nonerror_runs = nonerror_runs_by_p[p]
    data_size = n_nonerror_runs + len(df)

    nonprot_overheads = compute_overheads(solve_iterations[:data_size], 0)
    nonprot_overheads_by_p.append(nonprot_overheads)

nonprot_df = make_p_overhead_dataframe(nonprot_overheads_by_p)

In [ ]:
prot_overheads_by_p = []

for p in ps:
    # need to pad dataset to add non-error runs
    n_nonerror_runs = nonerror_runs_by_p[p]
    data_size = n_nonerror_runs + len(df)
    
    # for the purpose of choosing solve_iterations or errorfree_iterations, did_protect will
    # always be False for nonerror runs, but this is fine because n_protections is computed
    # later for the purposes of calculating overhead
    protections = np.append(df["prot_score"] > (1 + (1 / p)), [False] * n_nonerror_runs)

    prot_iterations = protect(solve_iterations[:data_size], protections)
    n_protections = (df_preds["output"] > (1 + (1 / p))).sum()

    prot_overheads = compute_overheads(prot_iterations, n_protections)
    prot_overheads_by_p.append(prot_overheads)

prot_df = make_p_overhead_dataframe(prot_overheads_by_p)

In [ ]:
random_overheads = []
for perc in percentages:
    protections = np.random.rand(len(df)) < perc
    prot_iterations = protect(solve_iterations[:len(df)], protections)
    n_protections = int(errorfree_iterations * n_rows * perc)
    prot_overheads = compute_overheads(prot_iterations, n_protections)
    random_overheads.append(prot_overheads.mean())
    
best_random_percentage = percentages[np.argmin(random_overheads)]
rand_overheads_by_p = []

for p in ps:
    # need to pad dataset to add non-error runs
    n_nonerror_runs = nonerror_runs_by_p[p]
    data_size = n_nonerror_runs + len(df)
    
    protections = np.random.rand(data_size) < best_random_percentage

    prot_iterations = protect(solve_iterations[:data_size], protections)
    n_protections = int(errorfree_iterations * n_rows * best_random_percentage)
    
    prot_overheads = compute_overheads(prot_iterations, n_protections)
    rand_overheads_by_p.append(prot_overheads)

rand_df = make_p_overhead_dataframe(rand_overheads_by_p)

In [ ]:
r2ns = np.array(list(pos_2norms.values()))
r2n_overheads = []
for perc in percentages:
    cutoff = np.quantile(r2ns, 1 - perc)
    protections = df["pos_2norm"] >= cutoff
    prot_iterations = protect(solve_iterations[:len(df)], protections)
    n_protections = errorfree_iterations * (r2ns >= cutoff).sum()
    prot_overheads = compute_overheads(prot_iterations, n_protections)
    r2n_overheads.append(prot_overheads.mean())

best_r2n_percentage = percentages[np.argmin(r2n_overheads)]
cutoff = np.quantile(r2ns, 1 - best_r2n_percentage)
r2n_overheads_by_p = []

for p in ps:
    # need to pad dataset to add non-error runs
    n_nonerror_runs = nonerror_runs_by_p[p]
    data_size = n_nonerror_runs + len(df)

    protections = np.append(df["pos_2norm"] >= cutoff, [False] * n_nonerror_runs)

    prot_iterations = protect(solve_iterations[:data_size], protections)
    n_protections = errorfree_iterations * (r2ns >= cutoff).sum()
    
    prot_overheads = compute_overheads(prot_iterations, n_protections)
    r2n_overheads_by_p.append(prot_overheads)

r2n_df = make_p_overhead_dataframe(r2n_overheads_by_p)

In [ ]:
plt.gcf().set_size_inches(4, 4)
plt.gcf().set_dpi(100)

sns.lineplot(nonprot_df, x="p", y="overhead",
             label="No Protection", c="red")
sns.lineplot(r2n_df, x="p", y="overhead",
             label=f"Best 2-Norm %", c="blue")
sns.lineplot(prot_df, x="p", y="overhead",
         label="Our Scheme", c="green")
# sns.lineplot(rand_df, x="p", y="overhead",
#              label=f"Best Random % ({best_random_percentage * 100}%)", c="blue")
# plt.plot(ps, [1] * len(ps), label="Full Protection (1)", c="black")

def formatter(x, pos):
    del pos
    return str(round(x * 100)) + "%"

plt.gca().yaxis.set_major_formatter(formatter)

plt.xlabel("$p$")
plt.ylabel("Mean Overhead")
plt.title(f"{matrix}", weight="bold")
plt.grid()
plt.xlim(0.01, 1)
plt.ylim(*ylims[matrix])
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 0.39), loc=(1.05, 0.25))
plt.savefig(f"./figures/{matrix}/mean_overheads.png", bbox_inches="tight")
plt.show()

In [ ]:
n_prots_per_p = []

for p in ps:
    n_protections = (df_preds["output"] > (1 + (1 / p))).sum()
    n_prots_per_p.append(n_protections)

s = pd.Series(n_prots_per_p)
plt.plot(ps, s)
plt.title("n_protections for each $p$")
plt.xlabel("$p$")
plt.ylabel("n_protections")
plt.savefig(f"./figures/{matrix}/n_protections_for_each_p.png")
plt.show()

In [ ]:
prot_avgs = prot_df.groupby("p").mean().rename(columns={"overhead": "prot_overhead"})
nonprot_avgs = nonprot_df.groupby("p").mean().rename(columns={"overhead": "nonprot_overhead"})
rand_avgs = rand_df.groupby("p").mean().rename(columns={"overhead": "rand_overhead"})
r2n_avgs = r2n_df.groupby("p").mean().rename(columns={"overhead": "r2n_overhead"})

avgs = prot_avgs.merge(nonprot_avgs, on="p").merge(rand_avgs, on="p").merge(r2n_avgs, on="p")
avgs.to_csv(f"./figures/{matrix}/mean_overheads.csv")
avgs